In [1]:
import pandas as pd
import matplotlib as pt
import requests
import yfinance as yf
import alpaca_trade_api as tradeapi
from tradingview_ta import TA_Handler, Interval

In [3]:
# Alpaca API config stuff

APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_MARKET_DATA = 'https://data.alpaca.markets/v1'
APCA_API_KEY = 'PKDQTHU4Z3R8ZTXFOGXN'
APCA_SECRET_KEY = 'FBE3WStDzqd6WFATRUVQbDLRlht0E6p7kO6gLOdB'

api = tradeapi.REST(APCA_API_KEY, APCA_SECRET_KEY, APCA_API_BASE_URL)

In [4]:
# TradingView Wrapper config stuff

short_handler = TA_Handler()
short_handler.set_interval_as(Interval.INTERVAL_1_WEEK)
short_handler.set_exchange_as_crypto_or_stock('NASDAQ')
short_handler.set_screener_as_stock('america')

long_handler = TA_Handler()
long_handler.set_interval_as(Interval.INTERVAL_1_MONTH)

In [5]:
# Finnhub API config stuff
# Real API Key: bvlaifv48v6sqkppd2e0
# Sandbox API Key: sandbox_bvlaifv48v6sqkppd2eg

# fib = finnhub.Client(api_key='bvlaifv48v6sqkppd2e0')

# IEX API config stuff

# IEX_CLOUD_API_KEY = 'Tpk_12e6eee6ed7d4026a0a87dee063b86bd'
# IEX_CLOUD_SECRET_KEY = 'Tsk_620c4eafd98f479f990e532c3a0b2cb0'
# SANDBOX_BASE_URL = 'https://sandbox.iexapis.com/stable/'

In [6]:
# 50-day SMA

def fifty_day_sma(ticker):
    FIFTY = 50.
    curr_date = str(datetime.now()).split()[0]
    year_ago = str(datetime.now() - timedelta(days=365)).split()[0]
    ticker_info = yf.Ticker(ticker)
 
    one_year_price_history = ticker_info.history(start=year_ago, end=curr_date)
    fifty_day_moving_average = one_year_price_history.iloc[-50:]['Close'].sum() / FIFTY
    return fifty_day_moving_average

In [7]:
# 200-day SMA

def two_hundred_day_sma(ticker):
    TWO_HUNDRED = 200.
    curr_date = str(datetime.now()).split()[0]
    year_ago = str(datetime.now() - timedelta(days=365)).split()[0]
    ticker_info = yf.Ticker(ticker)

    one_year_price_history = ticker_info.history(start=year_ago, end=curr_date)
    two_hundred_day_moving_average = one_year_price_history.iloc[-200:]['Close'].sum() / TWO_HUNDRED
    return two_hundred_day_moving_average

In [8]:
# checks if there is a Golden Cross (50-day SMA crosses 200-day SMA)

def golden_cross(ticker):
    fifty_sma = fifty_day_sma(ticker)
    two_hundred_sma = two_hundred_day_sma(ticker)
    
    if fifty_sma > two_hundred_sma:
        return True
    return False

In [9]:
# test SMAs + Golden Cross
# print(fifty_day_sma('PLTR'), two_hundred_day_sma('PLTR'), golden_cross('PLTR'))

In [10]:
# calculates 14-day MFI for specified stock

def mfi(ticker):
    curr_date = str(datetime.now()).split()[0]
    fifty_days_ago = str(datetime.now() - timedelta(days=50)).split()[0]
    ticker_info = yf.Ticker(ticker)
    one_year_price_history = ticker_info.history(start=fifty_days_ago, end=curr_date)
    two_week_ph = one_year_price_history[-14:]
    
    two_week_ph = two_week_ph.assign(TP = lambda tp:(two_week_ph['High'] + two_week_ph['Low'] + two_week_ph['Close']) / 3.)
    two_week_ph = two_week_ph.assign(RMF = lambda rmf: two_week_ph['TP'] * two_week_ph['Volume'])

    positive_rmf = 0
    negative_rmf = 0
    
    for i in range(1, len(two_week_ph['TP'])):
        if two_week_ph['TP'].iloc[i] > two_week_ph['TP'].iloc[i - 1]:
            positive_rmf += two_week_ph['RMF'].iloc[i]
        else:
            negative_rmf += two_week_ph['RMF'].iloc[i]
        
#     two_week_ph = two_week_ph.assign(TP_Diff = [1 if x == 1 else -1 for x in tp_diff])
    
    mf_ratio = positive_rmf / negative_rmf
    
    money_flow_index = 100 - (100 / (1 + mf_ratio))

    return money_flow_index

In [11]:
# testing MFI

# print(mfi('AAPL'), mfi('MSFT'), mfi('TGT'), mfi('WDAY'), mfi('GME'))

In [12]:
# # 200-day EMA

# def macd(ticker):
# #     return fib.technical_indicator(symbol=ticker, resolution='30', _from=1601371800 , to=1606642200 , indicator='macd', seriestype='c')
# #     resp = requests.get('https://finnhub.io/api/v1/indicator?symbol=MRVL&resolution=D&from=1606642200&to=1609234200&indicator=ema&&timeperiod=1token=sandbox_bvlaifv48v6sqkppd2eg')
# #     return resp.json()
#     return finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=1583098857, to=1584308457, indicator='ema', indicator_fields={"timeperiod": 5})

In [13]:
# macd('MRVL')

In [14]:
# calculates Relative Strength Index for specified stock
# from datetime import timezone

#     print(finnhub_client.technical_indicator(ticker, resolution='W', _from=  indicator='RSI'))

# def rsi(ticker):
#     dt = datetime(2020, 12, 28)
#     start_timestamp = int(dt.replace(tzinfo=timezone.utc).timestamp())
#     dte = datetime(2020, 12, 29)
#     end_timestamp = int(dte.replace(tzinfo=timezone.utc).timestamp())
#     print(start_timestamp, end_timestamp)
    
#     print(finnhub_client.aggregate_indicator('MSFT', 'D'))
    
#     print(finnhub_client.technical_indicator(symbol="AAPL", resolution='D', _from=1583098857, to=1584308457, indicator='rsi', indicator_fields={"timeperiod": 3}))

In [15]:
# rsi('AAPL')

In [16]:
short_handler.set_symbol_as('MSFT')

msft = short_handler.get_analysis().indicators
msft

{'Recommend.Other': -0.09090909,
 'Recommend.All': 0.35454545,
 'Recommend.MA': 0.8,
 'RSI': 59.08447079,
 'RSI[1]': 59.34203245,
 'Stoch.K': 77.04422998,
 'Stoch.D': 71.75642151,
 'Stoch.K[1]': 73.74332004,
 'Stoch.D[1]': 65.29031228,
 'CCI20': 90.0472573,
 'CCI20[1]': 104.16742011,
 'ADX': 23.1872319,
 'ADX+DI': 24.79575978,
 'ADX-DI': 13.43675152,
 'ADX+DI[1]': 25.00965918,
 'ADX-DI[1]': 14.13281553,
 'AO': 10.20282029,
 'AO[1]': 9.56276147,
 'Mom': 19.95,
 'Mom[1]': 6.52,
 'MACD.macd': 6.63713594,
 'MACD.signal': 7.16768846,
 'Rec.Stoch.RSI': 0,
 'Stoch.RSI.K': 64.77005652,
 'Rec.WR': 0,
 'W.R': -20,
 'Rec.BBPower': 0,
 'BBPower': 14.09832342,
 'Rec.UO': 0,
 'UO': 59.89983581,
 'close': 222.42,
 'EMA5': 219.87050405,
 'SMA5': 219.122,
 'EMA10': 217.56011668,
 'SMA10': 217.582,
 'EMA20': 213.26465003,
 'SMA20': 214.579,
 'EMA30': 207.91787389,
 'SMA30': 211.55266667,
 'EMA50': 196.51865664,
 'SMA50': 195.8066,
 'EMA100': 172.13896022,
 'SMA100': 166.0717,
 'EMA200': 139.32443877,
 '

In [8]:
msft = yf.Ticker('MSFT')
print(msft.info)

{'zip': '98052-6399', 'sector': 'Technology', 'fullTimeEmployees': 163000, 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and deskt